<h3><center>Assignment-1 (N-gram model)</center></h3>
<center>Name - Bbiswabasu Roy<center>
<center>Roll - 19EC39007<center>

In [107]:
def preprocess(filename, N):
    with open(filename, 'r') as f:
        sentences = [line.strip() for line in f.readlines()]  # Get list of sentences from given file

    for i in range(len(sentences)):
        sentences[i] = "<s> " * max(1, N - 1) + sentences[i].replace("\n", "") + " </s>" # Add start and end of sentence tag

    tokens = ' '.join(sentences).split(" ")  # Get list of individual tokens
    
    dictionary = dict() # store dictionary of all words along with their counts
    for token in tokens:  # Count number of occurences of each token
        if token in dictionary:
            dictionary[token] += 1
        else:
            dictionary[token] = 1

    for i in range(len(tokens)):  # Mark tokens with single occurrence with <UNK>
        if dictionary[tokens[i]] == 1 and tokens[i] != "<UNK>":
            del dictionary[tokens[i]]
            tokens[i] = "<UNK>"
            if "<UNK>" in dictionary:
                dictionary["<UNK>"] += 1
            else:
                dictionary["<UNK>"] = 1

    return dictionary, tokens


In [108]:
def get_N_gram_counts(tokens, N):
    N_gram_counts = dict() # stores N-grams with their counts
   
    for i in range(len(tokens)-N+1):
        N_gram = tuple(tokens[i:i+N]) # the tuple of N tokens is used as key in the dictionary of N-grams
        if N_gram in N_gram_counts:
            N_gram_counts[N_gram] += 1
        else:
            N_gram_counts[N_gram] = 1

    return N_gram_counts


In [109]:
def laplace_smoothing(tokens, N, dictionary):
    N_gram_counts = get_N_gram_counts(tokens, N) # stores N-grams with their counts
    N_1_gram_counts = get_N_gram_counts(tokens, N-1) # stores (N-1)-grams with their counts

    distribution = {} # stores probability distribution of N-grams in the corpus
    for key,value in N_gram_counts.items():
        distribution[key] = (value+1)/(N_1_gram_counts[key[:-1]] + len(dictionary))

    return distribution


In [110]:
def build_model(tokens, N, dictionary):
    distribution = None
    if N == 1:
        distribution = {(key,): value / len(tokens)
                        for key, value in dictionary.items()}
    else:
        distribution = laplace_smoothing(tokens, N, dictionary)

    return distribution


In [111]:
from math import log,exp
from itertools import product

def handle_oov_tokens(N_gram, model, N):
    masks  = list(reversed(list(product((0,1), repeat=N)))) # bitmask to mark whether a token should be considered or replaced with <UNK>
    for mask in masks:
        modified_N_gram = []
        for i in range(N):
            if mask[i]==1:
                modified_N_gram.append(N_gram[i])
            else:
                modified_N_gram.append("<UNK>")
        modified_N_gram = tuple(modified_N_gram)
        if modified_N_gram in model:
            return modified_N_gram


def compute_perplexity(N, dictionary, model):
    _,test_tokens = preprocess("test.txt", N) # ignore dictionary of test corpus and get the tokens
    f=open("out.txt", "w")

    perplexity = 0
    for i in range(len(test_tokens)-N+1):
        N_gram = tuple(test_tokens[i:i+N])
        modified_N_gram = handle_oov_tokens(N_gram, model, N)
        probability = model[modified_N_gram]
        f.writelines(str(modified_N_gram)+str(probability)+"\n")
        perplexity -= log(probability)
    perplexity *= 1/len(test_tokens)
    perplexity = exp(perplexity)
    return perplexity


In [112]:
def generate_sentence(sentence, N, k, model, min_len, max_len):
    sentence = "<s> " * max(1, N-1) + sentence
    sentence = sentence.split(' ')
    for i in range(k):
        cur_sentence = sentence[:]
        log_probability = 0
        while cur_sentence[-1] != "</s>":
            prev_tokens = tuple([]) if N == 1 else tuple(cur_sentence[-N+1:])
            
            possible_tokens = ((N_gram[-1],prob) for N_gram,prob in model.items() if N_gram[:-1]==prev_tokens)
            
            not_allowed = ["<UNK>"] + (["</s>"] if len(cur_sentence)<=min_len else []) + cur_sentence
            possible_tokens = filter(lambda token: token[0] not in not_allowed, possible_tokens)
            possible_tokens = sorted(possible_tokens, key=lambda token: token[1], reverse=True)
            next_token, next_token_prob = "</s>", 1
            
            if len(possible_tokens) != 0:
                next_token, next_token_prob = possible_tokens[0 if len(cur_sentence)!=len(sentence) else i]
            
            cur_sentence.append(next_token)
            log_probability += log(next_token_prob)
            
            if len(cur_sentence) == max_len:
                cur_sentence.append("</s>")
        cur_sentence = ' '.join(cur_sentence)
        print(cur_sentence,"[", log_probability, "]")

In [115]:
print("Waiting for N to be entered ...")
N = int(input("Enter N of model : "))

print("Preprocessing of training file started ...")
dictionary, tokens = preprocess("train.txt", N)
print("Preprcessing of training file completed\nDictionary size =", len(dictionary))

print("Model training started for N =", N, " ...")
model = build_model(tokens, N, dictionary)
print("Model training completed")

print("Computing perplexity for test set ...")
perplexity = compute_perplexity(N, dictionary, model)
print("Perplexity of test set =", perplexity)

min_len = 10 # minimum length of generated sentence (can be less if oov tokens are given)
max_len = 20 # maximum length of generated sentence
print("Waiting for k to be entered ...")
k = int(input("Enter number of sentences to generate : "))
print("Waiting for incomplete sentence to be entered ...")
sentence = input("Enter incomplete sentence : ")
print("Generating top", k, "sentence along with their log(probability) values ...")
generate_sentence(sentence, N, k, model, min_len, max_len)


Waiting for N to be entered ...
Preprocessing of training file started ...
Preprcessing of training file completed
Dictionary size = 23505
Model training started for N = 3  ...
Model training completed
Computing perplexity for test set ...
Perplexity of test set = 1412.184231933923
Waiting for k to be entered ...
Waiting for incomplete sentence to be entered ...
Generating top 10 sentence along with their log(probability) values ...
<s> <s> the company said it has agreed to sell its shares in a statement </s> [ -70.99898265704948 ]
<s> <s> the bank of japan governor satoshi sumita said he was not disclosed </s> [ -74.77123372163064 ]
<s> <s> the us agriculture department said it has agreed to sell its shares in a statement </s> [ -87.87579001441279 ]
<s> <s> the issue is rated a 1 by moodys and aa minus from b plus </s> [ -100.1971470671244 ]
<s> <s> the new york stock exchange said it has agreed to sell its shares in a statement </s> [ -98.8549653236569 ]
<s> <s> the government has sa